In [1]:
from transformers import BertForMaskedLM
from transformers import BertConfig
from transformers import BertTokenizer
import datasets
import json
import sys


In [2]:
def tokenize_function(examples):
    return tokenizer(examples["text"])
config = BertConfig()
model = BertForMaskedLM(config)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',use_fast=True) 
model = BertForMaskedLM.from_pretrained('/home/wanzhipeng/deepincubation/bert_wiki/checkpoint-145000')
# model_checkpoint = "distilroberta-base"
# tokenizer = BertTokenizer.from_pretrained(model_checkpoint, use_fast=True)
# tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
# model = BertForMaskedLM.from_pretrained(model_checkpoint)


In [6]:
import torch
a = torch.randn([3,3,4])
a
torch.triu(a)
print(torch.triu(a))
torch.triu(a, diagonal=1)
torch.triu(a, diagonal=-1)

b = torch.randn(4, 6)
b
torch.triu(b, diagonal=1)
torch.triu(b, diagonal=-1)

tensor([[ 1.0594, -0.1679, -0.1557, -1.9913,  1.2222, -0.1535],
        [ 0.8664,  0.3704,  0.2489,  0.4679, -0.4451,  0.8060],
        [ 0.0000, -1.4458,  2.2982, -1.4787,  1.1293, -1.5805],
        [ 0.0000,  0.0000,  1.1288, -0.2951,  1.2854,  0.0497]])

In [3]:
from datasets import load_dataset
# dataset = load_dataset("cc_news", split="train")
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
tokenized_datasets["train"][1]


Found cached dataset wikitext (/home/wanzhipeng/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/wanzhipeng/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-a0aeeb4e4600b55f_*_of_00004.arrow
Loading cached processed dataset at /home/wanzhipeng/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-8491a52215aa65e0_*_of_00004.arrow
Loading cached processed dataset at /home/wanzhipeng/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-fe40a1f675cf94d1_*_of_00004.arrow


{'input_ids': [101, 1027, 11748, 4801, 4360, 11906, 3523, 1027, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [4]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


Loading cached processed dataset at /home/wanzhipeng/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-fc031ffbf5d3610d_*_of_00004.arrow
Loading cached processed dataset at /home/wanzhipeng/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-4ba82bf0833b067d_*_of_00004.arrow
Loading cached processed dataset at /home/wanzhipeng/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6b0c8a5650ec01b9_*_of_00004.arrow


In [5]:
print(lm_datasets)

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2199
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 18549
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1921
    })
})


In [6]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'the first game and follows the " nameless ", a penal military unit serving the nation of gallia during the second europan war who perform secret black operations and are pitted against the imperial unit " calamaty raven ". [SEP] [CLS] the game began development in 2010, carrying over a large portion of the work done on valkyria chronicles ii. while it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. character designer raita honjou and composer hitoshi sakimoto both returned from previous entries, along with valkyria chronicles ii director takeshi'

In [5]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
training_args = TrainingArguments(
    output_dir="bert_wiki",          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    logging_strategy="steps",
    overwrite_output_dir=True,      
    num_train_epochs=500,            # number of training epochs, feel free to tweak
    # per_device_train_batch_size=1000, # the training batch size, put it as high as your GPU memory fits
    # gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    # per_device_eval_batch_size=1000,  # evaluation batch size
    logging_steps=500,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=500,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=10000
)


model_name = "fuckthelife"
# training_args = TrainingArguments(
#     f"{model_name}-finetuned-wikitext2",
#     evaluation_strategy = "steps",
#     logging_strategy="steps",
#     learning_rate=2e-5,
#     weight_decay=0.01,
# )


trainer = Trainer(
    model=model.to("cuda"),
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)




In [8]:
print(lm_datasets["train"])
print(len(lm_datasets))


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 18549
})
3


In [12]:

trainer.train(resume_from_checkpoint=True)

/home/wanzhipeng/miniconda3/envs/dl/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/270 [00:00<?, ?it/s]

/home/wanzhipeng/miniconda3/envs/dl/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


In [6]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model=model.to("cpu"),tokenizer=tokenizer)
unmasker("Hello I'm a [MASK] model.")

[{'score': 0.47138166427612305,
  'token': 2944,
  'token_str': 'model',
  'sequence': "hello i'm a model model."},
 {'score': 0.17571479082107544,
  'token': 7966,
  'token_str': 'fool',
  'sequence': "hello i'm a fool model."},
 {'score': 0.052288252860307693,
  'token': 2217,
  'token_str': 'side',
  'sequence': "hello i'm a side model."},
 {'score': 0.039750780910253525,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello i'm a new model."},
 {'score': 0.016525108367204666,
  'token': 4190,
  'token_str': 'leg',
  'sequence': "hello i'm a leg model."}]

In [8]:


import math
model = model.to("cuda")
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")



Perplexity: 114.16
